# Exercise 4: Exact diagonalization



In [ ]:
%matplotlib inline
import numpy as np
from scipy import sparse as sp
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
Id = sp.eye(2, format="csr")
Sx = sp.csr_matrix(np.array([[0, 1], [1,  0]]))
Sz = sp.csr_matrix(np.array([[1, 0], [0, -1]]))


In [ ]:
from functools import reduce

def _sigma(j: int, L: int, op: sp.spmatrix) -> sp.spmatrix:
    return reduce(lambda A, B: sp.kron(A, B, format="csr"), [Id if i != j else op for i in range(L)])

def sigma_z_j(j: int, L: int) -> sp.spmatrix:
    return _sigma(j, L, Sz)

def sigma_x_j(j: int, L: int) -> sp.spmatrix:
    return _sigma(j, L, Sx)

def sigma_z(L: int) -> list[sp.spmatrix]:
    return [sigma_z_j(j, L) for j in range(L)]

def sigma_x(L: int) -> list[sp.spmatrix]:
    return [sigma_x_j(j, L) for j in range(L)]

In [ ]:
def gen_hamiltonian(sx: list[sp.spmatrix], sz: list[sp.spmatrix], g: float, J: float) -> sp.spmatrix:
    H = -g * sum(sz)
    for j in range(len(sx)):
        H += -J * sx[j - 1] * sx[j]
    return H

def gen_hamiltonian_L(L: int, g: float, J: float) -> sp.spmatrix:
    sx = sigma_x(L)
    sz = sigma_z(L)
    return gen_hamiltonian(sx=sx, sz=sz, g=g, J=J)

In [ ]:
assert (
    gen_hamiltonian_L(L=2, g=0.1, J=1).todense() 
    == 
    np.array([
        [-0.2,  0. ,  0. , -2.  ,],
        [ 0. ,  0. , -2. ,  0.  ,],
        [ 0. , -2. ,  0. ,  0.  ,],
        [-2. ,  0. ,  0. ,  0.2 ,],
    ])).all()

In [ ]:
def get_ground_state(H: sp.spmatrix) -> np.ndarray:
    vals, vecs = sp.linalg.eigsh(H, k=1, which="SA")
    return vecs.reshape(-1)

def get_ground_state_L(L: int, g: float, J: float) -> np.ndarray:
    H = gen_hamiltonian_L(L=L, g=g, J=J)
    return get_ground_state(H)

In [ ]:
def largest_distance_spin_spin_correlation_L(L: int, g: float, J: float) -> float:
    sx = sigma_x(L)
    sz = sigma_z(L)
    return largest_distance_spin_spin_correlation(sx=sx, sz=sz, g=g, J=J)

def largest_distance_spin_spin_correlation(sx: list[sp.spmatrix], sz: list[sp.spmatrix], g: float, 
                                           J: float) -> float:
    L = len(sx)
    H = gen_hamiltonian(sx=sx, sz=sz, g=g, J=J)
    psi0 = get_ground_state(H)
    op = sx[1] @ sx[L//2]
    return np.inner(psi0, op * psi0)

In [ ]:
# Profiling
# %prun largest_distance_spin_spin_correlation_L(10, 0.2, 1)

# Turns out, sp.kron returns BSR format by default.
# That is around 20x slower with L=10 than CSR


In [ ]:
# Correlation plot
J = 1
plt.figure()
gs = np.linspace(0, 2, 20)
for L in [6, 8, 10, 12, 14, 16, 18]:
    Cs = []
    sx = sigma_x(L)
    sz = sigma_z(L)
    for g in tqdm(gs, desc=f"{L = }", total=gs.size):
        C = largest_distance_spin_spin_correlation(sx=sx, sz=sz, g=g, J=J)
        Cs.append(C)
    plt.plot(gs, Cs, label=f"{L = }")
plt.xlabel("g")
plt.ylabel("C")
plt.title("Largest-distance spin-spin correlation")
plt.legend()
plt.show()

In [ ]:
# Exited states vs g
# With large g, the system will be in a more disordered state.
# As such, the energy required to exite the system should be lower with larger g
J = 1
fig, (ax1, ax2) = plt.subplots(1, 2)
gs = np.linspace(0, 2, 20)
for L in [8, 10, 12, 14, 16]:
    E1 = []
    E2 = []
    for g in tqdm(gs, desc=f"{L = }", total=gs.size):
        H = gen_hamiltonian_L(L=L, g=g, J=J)
        eigenenergies, _ = sp.linalg.eigsh(H, k=3, which="SA")
        E1.append(eigenenergies[1] - eigenenergies[0])
        E2.append(eigenenergies[2] - eigenenergies[0])
    ax1.plot(gs, E1, label=f"{L = }")
    ax2.plot(gs, E2, label=f"{L = }")
ax1.set_xlabel("g")
ax1.set_ylabel("E")
ax1.set_title("First exited state")
ax1.legend()
ax2.set_xlabel("g")
ax2.set_ylabel("E")
ax2.set_title("Second exited state")
ax2.legend()
fig.tight_layout()
fig.show()